#　地図と震度の大きさ

In [36]:
pip install pandas requests matplotlib folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import folium

In [38]:

USGS_API_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.geojson" 


response = requests.get(USGS_API_URL)
data = response.json()


earthquake_features = data['features']
earthquake_data = []

for feature in earthquake_features:
   
    mag = feature['properties']['mag']
    place = feature['properties']['place']
    time = feature['properties']['time']
    
    lon = feature['geometry']['coordinates'][0]
    lat = feature['geometry']['coordinates'][1]
    
    earthquake_data.append({
        'magnitude': mag,
        'place': place,
        'time': time,
        'latitude': lat,
        'longitude': lon
    })


df = pd.DataFrame(earthquake_data)


df['time'] = pd.to_datetime(df['time'], unit='ms')


print(df.head())

   magnitude                              place                    time  \
0        5.4          111 km E of Miyako, Japan 2025-11-12 00:29:22.826   
1        5.1      131 km ENE of Tual, Indonesia 2025-11-11 21:23:31.013   
2        4.6  11 km SSW of Kosonsoy, Uzbekistan 2025-11-11 17:32:35.410   
3        4.7       242 km N of Fais, Micronesia 2025-11-11 15:34:53.216   
4        4.6          south of the Fiji Islands 2025-11-11 15:12:52.980   

   latitude  longitude  
0   39.8005   143.2201  
1   -5.3428   133.9024  
2   41.1625    71.4781  
3   11.9551   140.5932  
4  -25.0832   179.8323  


In [39]:
import pandas as pd
import requests


USGS_API_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.geojson" 


response = requests.get(USGS_API_URL)
data = response.json()

earthquake_features = data['features']
earthquake_data = []

for feature in earthquake_features:
    
    mag = feature['properties']['mag']
    place = feature['properties']['place']
    time = feature['properties']['time']
    
   
    coordinates = feature['geometry']['coordinates']
    lon = coordinates[0]
    lat = coordinates[1]
    
    depth = coordinates[2]
    
    earthquake_data.append({
        'magnitude': mag,
        'place': place,
        'time': time,
        'latitude': lat,
        'longitude': lon,
        'depth': depth 
    })


df = pd.DataFrame(earthquake_data)


df['time'] = pd.to_datetime(df['time'], unit='ms')


print(df.head())

   magnitude                              place                    time  \
0        5.4          111 km E of Miyako, Japan 2025-11-12 00:29:22.826   
1        5.1      131 km ENE of Tual, Indonesia 2025-11-11 21:23:31.013   
2        4.6  11 km SSW of Kosonsoy, Uzbekistan 2025-11-11 17:32:35.410   
3        4.7       242 km N of Fais, Micronesia 2025-11-11 15:34:53.216   
4        4.6          south of the Fiji Islands 2025-11-11 15:12:52.980   

   latitude  longitude    depth  
0   39.8005   143.2201   35.000  
1   -5.3428   133.9024   10.000  
2   41.1625    71.4781   19.892  
3   11.9551   140.5932   44.510  
4  -25.0832   179.8323  503.766  


In [40]:

def get_depth_category(depth):
    if depth <= 30:
        return 'Shallow (High Impact)' 
    elif depth <= 70:
        return 'Intermediate'         
    else:
        return 'Deep (Low Impact)'   


df['Impact_Category'] = df['depth'].apply(get_depth_category)

print("Impact カテゴリ別の件数:")

print(df['Impact_Category'].value_counts())

Impact カテゴリ別の件数:
Impact_Category
Shallow (High Impact)    289
Intermediate             145
Deep (Low Impact)        128
Name: count, dtype: int64


In [41]:
import folium



color_map = {
    'Shallow (High Impact)': 'red',    
    'Intermediate': 'orange',          
    'Deep (Low Impact)': 'blue'        
}


center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()


m = folium.Map(location=[center_lat, center_lon], zoom_start=2, tiles='CartoDB Positron')


for index, row in df.iterrows():

    radius = row['magnitude'] ** 2 * 0.5 
    
    
    color = color_map[row['Impact_Category']]
    
    
    popup_text = f"Magnitude: {row['magnitude']:.1f}<br>Place: {row['place']}<br>Depth: {row['depth']:.1f} km<br>Time: {row['time']}"

    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=popup_text
    ).add_to(m)


m

In [42]:
import os


output_dir = 'Week6'
map_output_file = os.path.join(output_dir, 'earthquake_impact_map.html')


if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"フォルダ '{output_dir}' を作成しました。")


m.save(map_output_file)

print(f"地図が {map_output_file} に保存されました。")

地図が Week6\earthquake_impact_map.html に保存されました。
